# LLM aided Decision Making in MCDA

openai api key is needed

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from openai import OpenAI
import ipywidgets as widgets
from IPython.display import display, clear_output
import os


file_path = 'Your Path of the KPI file'
data = pd.read_csv(file_path)

# data Normalization
scaler = MinMaxScaler()
normalized_data = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)

# initialize weights
initial_weights = {column: 1/len(normalized_data.columns) for column in normalized_data.columns}

# instance creation client
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "YOUR OPENAI API KEY"))

# weighted sum
def calculate_weighted_sum(data, weights):
    weighted_sum = np.dot(data, np.array([weights[col] for col in data.columns]))
    return weighted_sum

# iteration updates
def update_ranking(weights):
    # call weighted sum functions
    scores = calculate_weighted_sum(normalized_data, weights)
    
    # add scores to dataframe
    ranked_data = normalized_data.copy()
    ranked_data['Score'] = scores
    
    # ranking
    ranked_data = ranked_data.sort_values(by='Score', ascending=False)
    
    # Top 5 configurations
    with out:
        clear_output(wait=True)
        print("Top 5 configurations based on current weights:")
        display(ranked_data.head())
        
        # radar map
        create_radar_chart(ranked_data.iloc[0].drop('Score').values, normalized_data.columns.tolist(), 
                           baseline=baseline)

# GPT functions for weight adjustment
def adjust_weights_with_gpt4(weights, top_configurations, radar_data):
    radar_info = "Current radar chart data:\n"
    for kpi, values in radar_data.items():
        radar_info += f"{kpi}: Current Value: {values['current']}, Baseline: {values['baseline']}\n"
    
    prompt = (
        f"Given the following top configurations and their current weights:\n{top_configurations}\n\n"
        f"The current weights are:\n{weights}\n\n"
        f"{radar_info}\n"
        "Please suggest new weights for each KPI to improve the configuration performance on environmental impacts. "
        "Imagine three different problem-solving experts are answering this question, and five judgment experts are voting for each step. Assign the weights to each KPI based on the experts' voting. "
        "Return the weights in the following exact format (no explanations): 'KPI1: weight1, KPI2: weight2, ...'. "
        "Ensure all KPIs are included, and the weights sum up to 1."
    )
    
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a data optimization expert."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=150,
        temperature=0.7
    )
    
    gpt_response = response.choices[0].message.content.strip()
    print(f"GPT-4 Response: {gpt_response}")
    
    try:
        gpt_response = gpt_response.replace("'", "").replace(" ", "")
        new_weights = {}
        for item in gpt_response.split(','):
            kpi, weight = item.split(':')
            new_weights[kpi.strip()] = float(weight.strip())
        
        if set(new_weights.keys()) != set(weights.keys()):
            print("Incomplete response from GPT-4, using original weights.")
            return weights
        
    except Exception as e:
        print(f"Failed to parse GPT-4 response: {e}, using original weights.")
        new_weights = weights

    return new_weights

# weights iteration optimization
def iterative_weight_optimization(weights, iterations=5):
    for i in range(iterations):
        print(f"Iteration {i+1}")
        
        # Calculate scores and ranking
        scores = calculate_weighted_sum(normalized_data, weights)
        ranked_data = normalized_data.copy()
        ranked_data['Score'] = scores
        ranked_data = ranked_data.sort_values(by='Score', ascending=False)
        
        # Get top 5 configurations for feedback to GPT-4
        top_configurations = ranked_data.head().to_string()
        
        # Prepare radar chart data for GPT-4
        top_data_point = ranked_data.iloc[0].drop('Score').values
        radar_data = {
            label: {"current": top_data_point[i], "baseline": baseline[i]}
            for i, label in enumerate(normalized_data.columns)
        }
        
        # Update weights using GPT-4
        weights = adjust_weights_with_gpt4(weights, top_configurations, radar_data)
        
        # Update ranking and display results
        update_ranking(weights)
    
    return weights


# radar chart generation function
def create_radar_chart(data_point, labels, baseline, title='Radar Map'):
    num_vars = len(labels)
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

    data_point = data_point.tolist()
    data_point += data_point[:1]
    ax.plot(angles, data_point, linewidth=2, linestyle='solid', label='Selected Data')
    ax.fill(angles, data_point, alpha=0.25)

    baseline = baseline.tolist()
    baseline += baseline[:1]
    ax.plot(angles, baseline, color='red', linewidth=2, linestyle='dashed', label='Baseline')
    ax.fill(angles, baseline, color='red', alpha=0.15)

    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels)
    ax.set_yticklabels([])
    ax.set_ylim(min(min(data_point), min(baseline)), max(max(data_point), max(baseline)))

    plt.title(title, size=20, position=(0.5, 1.1))
    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    plt.show()


# display widgets and output
out = widgets.Output()

# display
display(out)

# initialization and optimization
final_weights = iterative_weight_optimization(initial_weights, iterations=5)
